# SGD Prompt Optimization Framework - 完整演示

本notebook演示如何使用SGD优化框架来优化RLAIF Judge Prompt。

## 目录
1. 环境设置
2. 加载数据和初始Prompt
3. 配置OpenAI API
4. 创建训练器
5. 执行训练
6. 分析结果

## 1. 环境设置

首先安装必要的依赖并设置环境变量。

In [ ]:
# 安装依赖
!pip install openai numpy scipy -q

In [ ]:
import os
import json
from pathlib import Path

# 设置环境变量
# 请在这里填写您的OpenAI API配置
os.environ['OPENAI_API_KEY'] = 'your-openai-api-key-here'  # 必需：您的OpenAI API密钥
os.environ['OPENAI_API_BASE'] = 'https://api.openai.com/v1'  # 可选：API端点（默认为OpenAI官方）
os.environ['OPENAI_MODEL'] = 'gpt-4'  # 可选：使用的模型（默认gpt-4）

# 数据文件路径
os.environ['PROMPT_PATH'] = 'demo_judge_prompt.json'
os.environ['DATASET_PATH'] = 'demo_dataset.jsonl'

print("✓ 环境变量设置完成")
print(f"  API Base: {os.environ.get('OPENAI_API_BASE')}")
print(f"  Model: {os.environ.get('OPENAI_MODEL')}")
print(f"  Prompt Path: {os.environ.get('PROMPT_PATH')}")
print(f"  Dataset Path: {os.environ.get('DATASET_PATH')}")

## 2. 准备数据和初始Prompt

### 2.1 创建示例数据集

我们将创建一个包含问答对及人工评分的数据集。

In [ ]:
# 创建示例数据集（中文问答示例）
demo_data = [
    # 高质量回答
    {"prompt": "什么是机器学习？", "response": "机器学习是人工智能的一个分支，它使计算机系统能够通过经验自动改进性能，而无需明确编程。机器学习算法使用统计技术使计算机能够从数据中'学习'，逐步提高特定任务的性能。", "score": 9.0},
    {"prompt": "解释什么是深度学习", "response": "深度学习是机器学习的一个子集，使用多层神经网络来学习数据的层次化表示。它受到人脑神经网络的启发，能够自动发现数据中的复杂模式和特征，在图像识别、自然语言处理等领域表现出色。", "score": 9.5},
    {"prompt": "Python有什么优势？", "response": "Python的主要优势包括：1）语法简洁易读，学习曲线平缓；2）拥有丰富的第三方库和框架；3）跨平台兼容性好；4）社区活跃，文档完善；5）在数据科学、AI、Web开发等领域应用广泛。", "score": 8.5},
    
    # 中等质量回答
    {"prompt": "什么是机器学习？", "response": "机器学习就是让计算机自己学习的技术。它可以从数据中找出规律，然后用这些规律来做预测或者分类。", "score": 6.0},
    {"prompt": "解释什么是深度学习", "response": "深度学习是一种人工智能技术，使用神经网络来处理数据。它在很多领域都有应用，比如图像识别和语音识别。", "score": 5.5},
    {"prompt": "Python有什么优势？", "response": "Python比较简单，容易学。而且有很多库可以用，做数据分析很方便。", "score": 5.0},
    
    # 低质量回答
    {"prompt": "什么是机器学习？", "response": "就是机器学习啊，很复杂的。", "score": 2.0},
    {"prompt": "解释什么是深度学习", "response": "深度学习就是学得很深。", "score": 1.5},
    {"prompt": "Python有什么优势？", "response": "不知道，没用过。", "score": 1.0},
    
    # 更多样本以增加训练数据
    {"prompt": "什么是自然语言处理？", "response": "自然语言处理（NLP）是人工智能的一个重要分支，专注于使计算机能够理解、解释和生成人类语言。它涉及文本分析、情感分析、机器翻译、问答系统等多个应用领域。", "score": 8.5},
    {"prompt": "解释什么是神经网络", "response": "神经网络是一种受生物神经系统启发的计算模型，由相互连接的节点（神经元）组成。每个连接都有权重，通过调整这些权重，网络可以学习从输入到输出的映射关系。", "score": 9.0},
    {"prompt": "什么是过拟合？", "response": "过拟合是指模型在训练数据上表现很好，但在新数据上表现较差的现象。这通常是因为模型过于复杂，记住了训练数据的噪声和细节，而没有学习到数据的真实模式。", "score": 8.0},
    {"prompt": "什么是自然语言处理？", "response": "NLP就是处理自然语言的技术，可以做翻译、文本分类什么的。", "score": 5.0},
    {"prompt": "解释什么是神经网络", "response": "神经网络就是模仿大脑的网络结构，用来做机器学习。", "score": 4.5},
    {"prompt": "什么是过拟合？", "response": "过拟合就是拟合过度了。", "score": 2.0},
    
    # 额外的高质量样本
    {"prompt": "什么是强化学习？", "response": "强化学习是机器学习的一个领域，通过智能体与环境的交互来学习最优策略。智能体在环境中执行动作，根据获得的奖励或惩罚来调整行为，目标是最大化长期累积奖励。典型应用包括游戏AI、机器人控制等。", "score": 9.0},
    {"prompt": "解释什么是卷积神经网络", "response": "卷积神经网络（CNN）是一种专门用于处理具有网格结构数据的深度学习模型，特别适合图像处理。它通过卷积层提取局部特征，通过池化层降低维度，能够自动学习图像的层次化特征表示。", "score": 8.5},
    {"prompt": "什么是迁移学习？", "response": "迁移学习是一种机器学习方法，将在一个任务上学到的知识应用到另一个相关任务上。例如，在大规模图像数据集上预训练的模型可以被微调用于特定的图像分类任务，大大减少了所需的训练数据和时间。", "score": 8.0},
    {"prompt": "什么是强化学习？", "response": "强化学习就是通过奖励来学习的方法。", "score": 3.5},
    {"prompt": "解释什么是卷积神经网络", "response": "CNN是处理图像的神经网络。", "score": 3.0},
    {"prompt": "什么是迁移学习？", "response": "迁移学习就是把学到的东西迁移到别的地方用。", "score": 2.5},
]

# 保存为JSONL格式
dataset_path = os.environ.get('DATASET_PATH', 'demo_dataset.jsonl')
with open(dataset_path, 'w', encoding='utf-8') as f:
    for item in demo_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"✓ 创建了包含 {len(demo_data)} 个样本的数据集")
print(f"  保存到: {dataset_path}")
print(f"\n示例数据:")
print(json.dumps(demo_data[0], ensure_ascii=False, indent=2))

### 2.2 创建初始JudgePrompt

定义初始的评分提示词，使用meta_sections模型。

In [ ]:
# 创建初始JudgePrompt（中文版本）
initial_prompt_data = {
    "sections": {
        "评分标准": """请根据以下维度评估回答质量：
1. 准确性：信息是否正确、可靠
2. 完整性：是否全面回答了问题
3. 清晰度：表达是否清晰易懂
4. 深度：解释是否深入、有见地

请综合考虑以上维度给出评分。""",
        "评分量表": """使用1-10分制评分：
- 1-2分：回答严重错误或完全不相关
- 3-4分：回答基本但不完整或有明显错误
- 5-6分：回答正确但缺乏深度或不够全面
- 7-8分：回答准确、完整且表达清晰
- 9-10分：回答优秀，准确、全面、深入且表达出色""",
        "输出格式": "请仅输出1-10之间的数字分数，可以是整数或小数（如7.5）。不要输出其他说明文字。"
    },
    "meta_sections": ["评分量表", "输出格式"]  # 这两个section在优化过程中不会被修改
}

prompt_path = os.environ.get('PROMPT_PATH', 'demo_judge_prompt.json')
with open(prompt_path, 'w', encoding='utf-8') as f:
    json.dump(initial_prompt_data, f, ensure_ascii=False, indent=2)

print(f"✓ 创建了初始JudgePrompt")
print(f"  保存到: {prompt_path}")
print(f"\n初始Prompt结构:")
print(f"  总sections数: {len(initial_prompt_data['sections'])}")
print(f"  Meta sections (不可修改): {initial_prompt_data['meta_sections']}")
print(f"  Editable sections (可优化): {[k for k in initial_prompt_data['sections'].keys() if k not in initial_prompt_data['meta_sections']]}")
print(f"\nPrompt内容:")
print(json.dumps(initial_prompt_data, ensure_ascii=False, indent=2))

## 3. 加载数据和Prompt

使用框架提供的工具加载数据和Prompt。

In [ ]:
from judge_prompt import JudgePrompt
from dataset_loader import DatasetLoader

# 加载JudgePrompt
print("1. 加载JudgePrompt...")
initial_prompt = JudgePrompt.load(prompt_path)
print(f"   ✓ 加载成功")
print(f"   - 总sections: {len(initial_prompt.sections)}")
print(f"   - Meta sections: {initial_prompt.meta_sections}")
print(f"   - Editable sections: {initial_prompt.get_editable_sections()}")

# 加载数据集
print("\n2. 加载数据集...")
loader = DatasetLoader()
prompts, responses, scores = loader.load_dataset(dataset_path)
print(f"   ✓ 加载了 {len(responses)} 个样本")

# 划分训练集和验证集
print("\n3. 划分训练集和验证集...")
train_resp, train_scores, val_resp, val_scores = loader.split_dataset(
    responses, scores, val_split=0.2, random_seed=42
)
print(f"   ✓ 训练集: {len(train_resp)} 样本")
print(f"   ✓ 验证集: {len(val_resp)} 样本")

print("\n数据统计:")
import numpy as np
print(f"  训练集分数范围: {np.min(train_scores):.1f} - {np.max(train_scores):.1f}")
print(f"  训练集平均分数: {np.mean(train_scores):.2f}")
print(f"  验证集分数范围: {np.min(val_scores):.1f} - {np.max(val_scores):.1f}")
print(f"  验证集平均分数: {np.mean(val_scores):.2f}")

## 4. 配置OpenAI API和创建LLM函数

使用OpenAI API创建三个LLM函数：Judge、Gradient Agent和Optimizer。

**重要**: 这些函数将使用真实的OpenAI API，会产生API调用费用。

In [ ]:
from openai_llm import create_openai_llm_functions

# 检查API密钥
api_key = os.environ.get('OPENAI_API_KEY')
if not api_key or api_key == 'your-openai-api-key-here':
    print("⚠️  警告: OPENAI_API_KEY 未设置或使用默认值")
    print("   请在上面的环境设置单元格中设置您的真实API密钥")
    raise ValueError("需要有效的OpenAI API密钥才能继续")

# 创建OpenAI LLM函数
print("创建OpenAI LLM函数...")
model = os.environ.get('OPENAI_MODEL', 'gpt-4')
judge_fn, gradient_fn, optimizer_fn = create_openai_llm_functions(
    model=model,
    judge_temperature=0.3,      # Judge需要一致性，使用较低温度
    gradient_temperature=0.7,   # Gradient Agent需要分析能力，使用中等温度
    optimizer_temperature=0.5   # Optimizer需要创造性但不过度，使用适中温度
)

print(f"✓ OpenAI LLM函数创建成功")
print(f"  Model: {model}")
print(f"  Judge Temperature: 0.3")
print(f"  Gradient Temperature: 0.7")
print(f"  Optimizer Temperature: 0.5")
print("\n⚠️  提示: 训练过程将调用OpenAI API，会产生费用")

## 5. 配置和创建训练器

配置训练参数并创建SGD Prompt Trainer。

In [ ]:
from trainer import SGDPromptTrainer

# 训练配置
# 注意：为了演示，我们使用较少的训练步数。实际应用中可以增加max_steps
training_config = {
    'max_steps': 15,           # 最大训练步数（演示用，实际可设为50-100）
    'batch_size': 8,           # 每步采样的batch大小
    'initial_lr': 0.1,         # 初始学习率
    'min_lr': 0.01,            # 最小学习率
    'warmup_steps': 3,         # 学习率warmup步数
    'alpha': 1.0,              # MAE损失权重
    'beta': 1.0,               # Rank损失权重
    'rank_loss_type': 'kendall',  # 排序损失类型: 'pairwise', 'kendall', 'spearman'
    'base_char_limit': 300,    # 基础字符限制（随学习率缩放）
    'structural_edit_threshold_ratio': 0.5,  # 结构编辑阈值比率
    'logging_steps': 1,        # 每N步打印日志
    'eval_steps': 1,           # 每N步进行完整评估
    'max_workers': 10,         # 并发LLM调用线程数
    'patience': 5,             # Early stopping的patience
    'enable_version_control': False,  # 是否启用Git版本控制（演示中关闭）
}

print("训练配置:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

# 创建训练器
print("\n创建SGD Prompt Trainer...")
trainer = SGDPromptTrainer(
    judge_llm_fn=judge_fn,
    gradient_llm_fn=gradient_fn,
    optimizer_llm_fn=optimizer_fn,
    initial_prompt=initial_prompt,
    train_responses=train_resp,
    train_human_scores=train_scores,
    val_responses=val_resp,
    val_human_scores=val_scores,
    config=training_config
)

print("✓ 训练器创建成功")

## 6. 执行训练

开始SGD优化训练过程。这将：
1. 使用当前Prompt对训练样本进行评分
2. 计算损失（MAE + Ranking Loss）
3. 构建代理梯度
4. 生成Prompt修改建议
5. 应用修改并验证

**注意**: 这个过程会调用OpenAI API，请确保有足够的API额度。

In [ ]:
print("="*70)
print("开始训练...")
print("="*70)
print()

# 执行训练
best_prompt = trainer.train()

print()
print("="*70)
print("训练完成！")
print("="*70)

## 7. 分析训练结果

查看训练历史和最优Prompt。

In [ ]:
# 获取训练历史
history = trainer.get_history()

print("\n训练历史摘要:")
print(f"  总步数: {len(history)}")
print(f"  最佳验证损失: {trainer.best_val_loss:.4f}")
print(f"  最佳步数: {trainer.best_step}")

# 显示每步的关键指标
print("\n详细训练历史:")
print("-" * 90)
print(f"{'Step':<6} {'LR':<8} {'Train Loss':<12} {'Val Loss':<12} {'Kendall τ':<12} {'Modified':<10}")
print("-" * 90)
for step_info in history:
    step = step_info.get('step', 0)
    lr = step_info.get('learning_rate', 0)
    train_loss = step_info.get('train_loss', 0)
    val_loss = step_info.get('val_loss', 0)
    kendall = step_info.get('val_kendall_tau', 0)
    modified = '✓' if step_info.get('modification_valid', False) else '✗'
    
    print(f"{step:<6} {lr:<8.4f} {train_loss:<12.4f} {val_loss:<12.4f} {kendall:<12.4f} {modified:<10}")
print("-" * 90)

In [ ]:
# 绘制训练曲线
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']  # 支持中文显示
matplotlib.rcParams['axes.unicode_minus'] = False

steps = [h['step'] for h in history]
train_losses = [h['train_loss'] for h in history]
val_losses = [h.get('val_loss', 0) for h in history]
kendall_taus = [h.get('val_kendall_tau', 0) for h in history]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 损失曲线
axes[0].plot(steps, train_losses, 'b-', label='训练损失', marker='o')
axes[0].plot(steps, val_losses, 'r-', label='验证损失', marker='s')
axes[0].set_xlabel('训练步数')
axes[0].set_ylabel('损失')
axes[0].set_title('训练和验证损失')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Kendall τ曲线
axes[1].plot(steps, kendall_taus, 'g-', label='Kendall τ', marker='^')
axes[1].set_xlabel('训练步数')
axes[1].set_ylabel('Kendall τ相关系数')
axes[1].set_title('排序相关性')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ 训练曲线已绘制")

## 8. 比较初始Prompt和优化后的Prompt

In [ ]:
print("="*70)
print("初始 JudgePrompt:")
print("="*70)
print(initial_prompt.get_full_prompt())
print()

print("="*70)
print("优化后的 JudgePrompt:")
print("="*70)
print(best_prompt.get_full_prompt())
print()

# 比较sections
print("="*70)
print("Sections变化:")
print("="*70)
print(f"初始sections: {list(initial_prompt.sections.keys())}")
print(f"优化后sections: {list(best_prompt.sections.keys())}")
print()

# 新增的sections
new_sections = set(best_prompt.sections.keys()) - set(initial_prompt.sections.keys())
if new_sections:
    print(f"新增sections: {list(new_sections)}")
    for section in new_sections:
        print(f"\n  [{section}]:")
        print(f"  {best_prompt.sections[section]}")
else:
    print("没有新增sections")

# 删除的sections
removed_sections = set(initial_prompt.sections.keys()) - set(best_prompt.sections.keys())
if removed_sections:
    print(f"\n删除的sections: {list(removed_sections)}")
else:
    print("\n没有删除sections")

# 修改的sections
print("\n修改的sections:")
for section in set(initial_prompt.sections.keys()) & set(best_prompt.sections.keys()):
    if initial_prompt.sections[section] != best_prompt.sections[section]:
        print(f"\n  [{section}]:")
        print(f"  初始: {initial_prompt.sections[section][:100]}...")
        print(f"  优化: {best_prompt.sections[section][:100]}...")

## 9. 保存优化后的Prompt

In [ ]:
# 保存最佳Prompt
optimized_prompt_path = 'notebook_optimized_judge_prompt.json'
best_prompt.save(optimized_prompt_path)

print(f"✓ 优化后的Prompt已保存到: {optimized_prompt_path}")

# 保存训练历史
history_path = 'notebook_training_history.json'
trainer.save_history(history_path)

print(f"✓ 训练历史已保存到: {history_path}")

# 显示保存的文件内容
print("\n优化后Prompt的JSON格式:")
with open(optimized_prompt_path, 'r', encoding='utf-8') as f:
    print(f.read())

## 10. 测试优化效果

使用初始Prompt和优化后的Prompt对相同样本进行评分，比较效果。

In [ ]:
# 选择几个测试样本
test_indices = [0, 3, 6]  # 高、中、低质量各一个

print("测试样本评分对比:")
print("="*90)

for idx in test_indices:
    response = val_resp[idx]
    human_score = val_scores[idx]
    
    # 使用初始Prompt评分
    initial_judge_score = judge_fn(initial_prompt.get_full_prompt(), response)
    
    # 使用优化后Prompt评分
    optimized_judge_score = judge_fn(best_prompt.get_full_prompt(), response)
    
    print(f"\n样本 {idx + 1}:")
    print(f"  回答: {response[:80]}...")
    print(f"  人工评分: {human_score:.1f}")
    print(f"  初始Prompt评分: {initial_judge_score:.1f}")
    print(f"  优化后Prompt评分: {optimized_judge_score:.1f}")
    print(f"  初始误差: {abs(initial_judge_score - human_score):.2f}")
    print(f"  优化后误差: {abs(optimized_judge_score - human_score):.2f}")
    print("-" * 90)

print("\n✓ 测试完成")

## 总结

本notebook演示了完整的SGD Prompt优化流程：

1. ✅ 设置OpenAI API环境变量
2. ✅ 创建和加载数据集（JSONL格式）
3. ✅ 创建和加载初始JudgePrompt（JSON格式，使用meta_sections）
4. ✅ 配置OpenAI API的三个LLM函数（Judge、Gradient Agent、Optimizer）
5. ✅ 使用中文指令prompt进行训练
6. ✅ 完整的SGD优化训练循环
7. ✅ 可视化训练过程和结果
8. ✅ 比较优化前后的Prompt差异
9. ✅ 保存优化结果
10. ✅ 测试优化效果

### 关键特性

- **Meta Sections模型**: 使用`meta_sections`明确指定不可修改的sections（如评分量表、输出格式），其他sections自动可编辑
- **学习率调度**: 高LR阶段可以添加/删除sections，低LR阶段仅修改内容
- **真实OpenAI API**: 完全使用OpenAI API，无mock函数
- **中文支持**: 支持中文数据集和中文prompt
- **完整训练流程**: 包含数据加载、训练、验证、可视化、保存等完整环节

### 下一步

您可以：
- 使用自己的数据集（JSONL格式）
- 调整训练配置（增加步数、调整学习率等）
- 修改初始Prompt的sections
- 尝试不同的meta_sections配置
- 使用不同的OpenAI模型
